In [ ]:
# TODO : dev set for sequenced dataloader
# TODO : dev set that's a mix of trianing set instead of held out
# TODO : understand torch generator
# TODO : implement torch generator for trianing
# TODO : could we define param groups, one for the encoder, and put a lower learning rate? 
# i noticed unfrozen training loss is volatile

from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from lib.ekyn import *
from lib.env import *
import copy
from sage.utils import *

states = []
evals = []
# for model in range(1,5):
for model in range(24,25):
    MODEL_ID = f'{model}'

    state = torch.load(f'{TMP_EXPERIMENTS_PATH}/{MODEL_ID}/state.pt',map_location='cpu',weights_only=False)
    states.append(state)
    state['name'] = f'{model}'

df = pd.DataFrame(states)

df['best_dev_f1'] = df['devf1i'][0][df['best_dev_loss_epoch'].item()]
df['best_dev_f1_epoch'] = torch.tensor(df['devf1i'][0]).argmax().item()
df['dev_f1_at_best_dev_f1_epoch'] = df['devf1i'][0][df['best_dev_f1_epoch'].item()]
df['test_f1_at_best_dev_loss_epoch'] = df['testf1i'][0][df['best_dev_loss_epoch'].item()]
df['test_f1_at_best_dev_f1_epoch'] = df['testf1i'][0][df['best_dev_f1_epoch'].item()]

display(df.set_index(keys='name')[['fold','widthi','depthi','best_dev_loss','best_dev_f1','best_dev_loss_epoch','best_dev_f1_epoch','test_f1_at_best_dev_loss_epoch','test_f1_at_best_dev_f1_epoch']])

In [ ]:
fig,axes = plt.subplots(nrows=1,ncols=2,figsize=(16,4.2))

colors = ['C0','C1','C2','C3','C4']
for i,state in enumerate(df.iloc):
    axes[0].plot(state['trainlossi'],color=colors[i],linestyle='-',label='train')
    axes[0].plot(state['devlossi'],color=colors[i],linestyle=':',label='val')
    # axes[0].plot(state['testlossi'],color=colors[i],linestyle='--',label='test')
    axes[0].axvline(state['best_dev_loss_epoch'],color=colors[i],linestyle=':',)
    axes[0].axhline(state['best_dev_loss'],color=colors[i],linestyle=':',)
    # axes[0].axhline(state['testlossi'][state['best_dev_loss_epoch']],color=colors[i],linestyle='--')
plt.yscale('log')
axes[0].set_ylim([.15,.4])
axes[0].legend()
for i,state in enumerate(df.iloc):
    axes[1].plot(state['devf1i'],color=colors[i],linestyle=':')
    # axes[1].plot(state['testf1i'],color=colors[i],linestyle='--')
    axes[1].axvline(state['best_dev_loss_epoch'],color=colors[i],linestyle=':')
    axes[1].axhline(state['best_dev_f1'],color=colors[i],linestyle=':')
    # axes[1].axvline(state['best_dev_loss_epoch'],color=colors[i],linestyle=':')
    axes[1].axvline(state['best_dev_f1_epoch'],color=colors[i+1],linestyle=':')
    axes[1].axhline(state['dev_f1_at_best_dev_f1_epoch'],color=colors[i+1],linestyle=':')
    # axes[1].axvline(torch.tensor(state['devf1i']).argmax(),color=colors[i],linestyle='--')
    # axes[1].axhline(state['testf1i'][state['best_dev_loss_epoch']],color=colors[i],linestyle=':')
    # axes[1].axhline(state['testf1i'][torch.tensor(state['devf1i']).argmax()],color=colors[i],linestyle='--')

axes[0].set_title('loss')
axes[1].set_title('f1')
axes[0].set_xlabel('epoch')
axes[1].set_xlabel('epoch')
# plt.yscale('log')
# plt.ylim([.85,.95])
# plt.savefig('out.jpg')

In [ ]:
fig,axes = plt.subplots(nrows=1,ncols=2,figsize=(16,4.2))

colors = ['C0','C1','C2','C3','C4']
for i,state in enumerate(df.iloc):
    axes[0].plot(state['trainlossi'],color=colors[i],linestyle='-',label='train')
    axes[0].plot(state['devlossi'],color=colors[i],linestyle=':',label='val')
    # axes[0].plot(state['testlossi'],color=colors[i],linestyle='--',label='test')
    axes[0].axvline(state['best_dev_loss_epoch'],color=colors[i],linestyle=':',)
    axes[0].axhline(state['best_dev_loss'],color=colors[i],linestyle=':',)
    # axes[0].axhline(state['testlossi'][state['best_dev_loss_epoch']],color=colors[i],linestyle='--')
plt.yscale('log')
axes[0].set_ylim([.15,.4])
axes[0].legend()
for i,state in enumerate(df.iloc):
    axes[1].plot(state['devf1i'],color=colors[i],linestyle=':')
    # axes[1].plot(state['testf1i'],color=colors[i],linestyle='--')
    axes[1].axvline(state['best_dev_loss_epoch'],color=colors[i],linestyle=':')
    axes[1].axhline(state['best_dev_f1'],color=colors[i],linestyle=':')
    # axes[1].axvline(state['best_dev_loss_epoch'],color=colors[i],linestyle=':')
    axes[1].axvline(state['best_dev_f1_epoch'],color=colors[i+1],linestyle=':')
    axes[1].axhline(state['dev_f1_at_best_dev_f1_epoch'],color=colors[i+1],linestyle=':')
    # axes[1].axvline(torch.tensor(state['devf1i']).argmax(),color=colors[i],linestyle='--')
    # axes[1].axhline(state['testf1i'][state['best_dev_loss_epoch']],color=colors[i],linestyle=':')
    # axes[1].axhline(state['testf1i'][torch.tensor(state['devf1i']).argmax()],color=colors[i],linestyle='--')

axes[0].set_title('loss')
axes[1].set_title('f1')
axes[0].set_xlabel('epoch')
axes[1].set_xlabel('epoch')
# plt.yscale('log')
# plt.ylim([.85,.95])
# plt.savefig('out.jpg')

In [ ]:

df.widthi = df.widthi.astype(str)
df = df.reset_index()
df = df[df['fold'] == 0]
df.loc[:4,'type'] = 'encoder'
df.loc[4:,'type'] = 'lstm'
df.loc[:4,'encoder_name'] = df.loc[:4,'name']
df['encoder_experiment_name'] = df['encoder_experiment_name'].apply(lambda x: str(x).split('/')[-1] if not pd.isna(x) else x)
df.loc[4:,'encoder_name'] = df.loc[4:,'encoder_experiment_name']
df = df.drop(['encoder_experiment_name'],axis=1)

# Plot the data
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='encoder_name', y='best_dev_loss', hue='type')
plt.xlabel('Encoder Name')
plt.ylabel('Best Dev Loss')
plt.title('Comparison of Best Dev Loss between Encoders and Corresponding LSTMs')
plt.show()

In [ ]:

df['encoder_name'] = df['name']
display(df)
sns.barplot(data=df,x='fold',y='best_dev_loss',hue='widthi')
# # display(pd.DataFrame(states).set_index(keys='name')[['sequence_length','bidirectional','fold','frozen_encoder']])

In [ ]:
fig,axes = plt.subplots(nrows=1,ncols=2,figsize=(16,4.2))

colors = ['C0','C1','C2','C3','C4']
for i,state in enumerate(states):
    axes[0].plot(state['trainlossi'],color=colors[i],linestyle='-',label=state['name'])
    axes[0].plot(state['devlossi'],color=colors[i],linestyle=':')
    axes[0].plot(state['testlossi'],color=colors[i],linestyle='--')
    axes[0].axvline(state['best_dev_loss_epoch'],color=colors[i],linestyle=':',)
    axes[0].axhline(state['best_dev_loss'],color=colors[i],linestyle=':',)
    axes[0].axhline(state['testlossi'][state['best_dev_loss_epoch']],color=colors[i],linestyle='--')
plt.yscale('log')
axes[0].set_ylim([.15,.3])
axes[0].legend()
for i,state in enumerate(states):
    axes[1].plot(state['devf1i'],color=colors[i],linestyle=':')
    axes[1].plot(state['testf1i'],color=colors[i],linestyle='--')
    axes[1].axvline(state['best_dev_loss_epoch'],color=colors[i],linestyle=':')
    axes[1].axvline(torch.tensor(state['devf1i']).argmax(),color=colors[i],linestyle='--')
    axes[1].axhline(state['testf1i'][state['best_dev_loss_epoch']],color=colors[i],linestyle=':')
    axes[1].axhline(state['testf1i'][torch.tensor(state['devf1i']).argmax()],color=colors[i],linestyle='--')
# plt.yscale('log')
# plt.ylim([.85,.95])
# plt.savefig('out.jpg')

In [ ]:
evals = []
for state in states:
    model = copy.deepcopy(state['model'])
    model.load_state_dict(state['best_model_wts_dev_loss'])
    trainloader,testloader = get_sequenced_dataloaders_loo(**state)
    evals.append(evaluate(dataloader=testloader,model=model,criterion=state['criterion'],device='cuda'))

In [ ]:
for loss,y_true,y_pred in evals:
    fig,axes = plt.subplots(nrows=1,ncols=2,figsize=(8,4.5))
    ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='true',colorbar=False,ax=axes[0])
    ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='pred',colorbar=False,ax=axes[1])
    axes[0].set_title('Recall')
    axes[1].set_title('Precision')
    plt.suptitle(f'f1 : {f1_score(y_true,y_pred,average="macro"):.3f}   loss: {loss:.3f}')

In [ ]:
# df = pd.DataFrame(states).reset_index()
# # df = df.melt(id_vars='index',value_vars=['best_dev_loss','best_test_loss'])
# df = df.melt(id_vars='index',value_vars=['best_dev_loss'])
# sns.barplot(data=df,x='index',y='value',hue='variable')

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import ConfusionMatrixDisplay
# from lib.ekyn import *
# from lib.env import *
# import copy
# from sage.utils import *

# MODEL_ID = f'15'

# state = torch.load(f'{TMP_EXPERIMENTS_PATH}/{MODEL_ID}/state.pt',map_location='cpu',weights_only=False)
# model = copy.deepcopy(state['model'])
# model.load_state_dict(state['best_model_wts_dev_loss'])
# trainloader,testloader = get_sequenced_dataloaders_loo(**state)
# loss,y_true,y_pred = evaluate(dataloader=testloader,model=model,criterion=state['criterion'],device='cuda')
# fig,axes = plt.subplots(nrows=1,ncols=2,figsize=(8,4.5))
# ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='true',colorbar=False,ax=axes[0])
# ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='pred',colorbar=False,ax=axes[1])
# axes[0].set_title('Recall')
# axes[1].set_title('Precision')
# plt.suptitle(f'f1 : {f1_score(y_true,y_pred,average="macro"):.3f}   loss: {loss:.3f}')
# print(state['criterion'].weight)
# # TODO : add best loss epoch during training
# plot_loss(state,f'{TMP_EXPERIMENTS_PATH}/{MODEL_ID}')